# Example of ```VariableRowDiv```
Build a volatility skew application that shows multiple graphs using the class ```VariableRowDiv```

### Usage:
1. Run all of the cells below.  
2. On the final cell, the url ```http://127.0.0.1:8500``` will appear. 
3. Click on it and another web page will launch that displays the output of the app.

In [ ]:
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

import pandas as pd
import numpy as np
import traceback

from dashgrid import dash_df_components as ddfc


### Create a logger that can be used within all components

In [ ]:
logger = ddfc.dgc.init_root_logger()

### Generate some example data for the application from 3 csv files

In [ ]:
# location of 3 main csv files
PATH_DATA_HOME = './' 
FILENAME_SKEW = 'df_iv_skew_COMMOD.csv'
FILENAME_IV = 'df_iv_final_COMMOD.csv'
FILENAME_FUT = 'df_cash_futures_COMMOD.csv'

df_iv_skew = None
df_iv_final = None
df_cash_futures = None
for commod in ['CL','CB','ES','NG']:
    fn_skew = FILENAME_SKEW.replace('COMMOD',commod)
    df_skew = pd.read_csv(f'{PATH_DATA_HOME}/{fn_skew}')
    fn_iv = FILENAME_IV.replace('COMMOD',commod)
    df_iv = pd.read_csv(f'{PATH_DATA_HOME}/{fn_iv}')
    fn_fut = FILENAME_FUT.replace('COMMOD',commod)
    df_fut = pd.read_csv(f'{PATH_DATA_HOME}/{fn_fut}')
    df_skew['commod'] = commod
    df_iv['commod'] = commod
    df_fut['commod'] = commod
    if df_iv_skew is None:
        df_iv_skew = df_skew.copy()
        df_iv_final = df_iv.copy()
        df_cash_futures = df_fut.copy()
    else:
        df_iv_skew = df_iv_skew.append(df_skew.copy())
        df_iv_final = df_iv_final.append(df_iv.copy())
        df_cash_futures = df_cash_futures.append(df_fut.copy())

df_iv_skew = df_iv_skew.rename(columns={c:float(c) for c in df_iv_skew.columns.values if '0.' in c})


### Method to select main dataframes symbol (like CL, CB or ES) and by year

In [ ]:
def slice_dataframes(SYMBOL_TO_RESEARCH,df_iv_final_in,
                        df_iv_skew_in,df_cash_futures_in,
                        dist_from_zero=.1,year=None):
    # Step 00: select only SYMBOL_TO_RESEARCH, and for a specific year,  from DataFrames 
    df_iv_final = df_iv_final_in[df_iv_final_in.symbol.str.slice(0,2)==SYMBOL_TO_RESEARCH].copy()
    df_iv_skew = df_iv_skew_in[df_iv_skew_in.symbol.str.slice(0,2)==SYMBOL_TO_RESEARCH].copy()
    df_cash_futures = df_cash_futures_in[df_cash_futures_in.symbol.str.slice(0,2)==SYMBOL_TO_RESEARCH].copy()

    #  Now slice on specific year
    year = 'all' if year is None else year
    if str(year).lower() != 'all':
        y = int(str(year))
        beg_year = y*100*100+1*100+1
        end_year = y*100*100+12*100+31
        df_iv_final = df_iv_final[(df_iv_final.settle_date>=beg_year) & (df_iv_final.settle_date<=end_year)]
        df_iv_skew = df_iv_skew[(df_iv_skew.settle_date>=beg_year) & (df_iv_skew.settle_date<=end_year)]
        df_cash_futures = df_cash_futures[(df_cash_futures.settle_date>=beg_year) & (df_cash_futures.settle_date<=end_year)]

    # Step 01: create atm implied vol table, that also has the cash price for each settle_date
    df_atmv = df_iv_final[['settle_date','atm_iv']].drop_duplicates()
    df_cf = df_cash_futures[df_cash_futures.symbol==f'{SYMBOL_TO_RESEARCH}Z99']
    df_atmv = df_atmv.merge(df_cf[['settle_date','close']],how='inner',on='settle_date')
        
    # Step 02: create df_skew_2, which holds skew difference between 
    #   positive dist_from_zero and negative dist_from_zero, for each settle_date
    df_skew_2 = df_iv_skew.copy()
    df_skew_2.index.name = None
    skew_range_col = f'iv_skew'
    df_skew_2[skew_range_col] = df_skew_2[dist_from_zero] - df_skew_2[-dist_from_zero]
    df_skew_2.settle_date = df_skew_2.settle_date.astype(int)
    df_skew_2 = df_skew_2[['settle_date',skew_range_col]]
    
    return df_skew_2,df_atmv


### Method that shows skew vs atm vol by day

In [ ]:
def skew_vs_atm(SYMBOL_TO_RESEARCH,df_iv_final_in,df_iv_skew_in,df_cash_futures_in,
                     dist_from_zero=.1,year=None):

    df_skew_2,df_atmv = slice_dataframes(
        SYMBOL_TO_RESEARCH,df_iv_final_in,df_iv_skew_in,df_cash_futures_in,
        dist_from_zero=dist_from_zero,year=year)
    
    # Step 03: create atm vs close 
    chart_title_0 = f'{SYMBOL_TO_RESEARCH} atm vol vs close'
    df_atm_vs_close = df_atmv[['settle_date','atm_iv','close']]
    
    # Step 04: merge skew and atm vol/close tables
    df_ivs = df_skew_2.merge(df_atmv,how='inner',on='settle_date')
    df_ivs = df_ivs.sort_values('settle_date')
    
    # Step 05: plot skew vs atm_iv
    chart_title_1 = f'{SYMBOL_TO_RESEARCH} skew {dist_from_zero*100}% up and down vs atm vol'
    df_ivs_skew_vs_atm_iv = df_ivs[['settle_date',f'iv_skew','atm_iv']]
    
    # Step 06: plot skew vs close
    chart_title_2 = f'{SYMBOL_TO_RESEARCH} skew {dist_from_zero*100}% up and down vs close'
    df_ivs_skew_vs_close = df_ivs[['settle_date',f'iv_skew','close']]

    d1 = {'initial_data':df_ivs_skew_vs_atm_iv,'title':chart_title_1,'use_yaxis2':True}
    d2 = {'initial_data':df_ivs_skew_vs_close,'title':chart_title_2,'use_yaxis2':True}
    return d1,d2


### Method that shows atm vol vs close by day

In [ ]:
def atm_vs_close(SYMBOL_TO_RESEARCH,df_iv_final_in,df_iv_skew_in,df_cash_futures_in,year=None):
    _,df_atmv = slice_dataframes(
        SYMBOL_TO_RESEARCH,df_iv_final_in,df_iv_skew_in,df_cash_futures_in,year=year)
    
    # Step 02: plot atm_iv vs close
    chart_title = f'{SYMBOL_TO_RESEARCH} atm vol vs close'
    df_atm_vs_close = df_atmv[['settle_date','atm_iv','close']]
    
    return {'initial_data':df_atm_vs_close,'title':chart_title,'use_yaxis2':True}


### Build callback that returns plotly graph Figures

In [ ]:
# define a storage components
#  build a store here
SKEW_RANGE_LIST = [.05,.1,.2]
def create_figs_closure(df_iv_final_in, df_iv_skew_in,df_cash_futures_in):
    def _create_figs(input_dict):
#         print(f'_create_figs {input_dict}')
        none_return = None
        if input_dict is None or len(input_dict.keys())<2:
            return none_return
        sym_to_plot = input_dict['mdd2_0']
        if sym_to_plot is None:
            return none_return
        
        year_to_plot = input_dict['mdd2_1']
        year_to_plot = 'all' if year_to_plot is None else year_to_plot
        all_figs = []
        fig = atm_vs_close(sym_to_plot,df_iv_final_in,df_iv_skew_in,df_cash_futures_in,
                                year=year_to_plot)        
        all_figs.append([fig])
        for d in SKEW_RANGE_LIST:
            fig_skew_vs_atm,fig_skew_vs_price = skew_vs_atm(
                sym_to_plot,df_iv_final_in,df_iv_skew_in,
                df_cash_futures_in,dist_from_zero=d,year=year_to_plot)
            all_figs.append([fig_skew_vs_atm])
            all_figs.append([fig_skew_vs_price])
        return all_figs
    return _create_figs


### Create title div

In [ ]:
tm_text1 = '''
# Commodity Option Skew Analysis
## Select a Commodity, Year and Monthcode below to display charts showing:
'''
tm1 = ddfc.MarkdownLine(tm_text1,text_size=1)
tm_text2 = '''
* atm vol vs price
* skew vs price
* skew vs atm vol
'''
tm2 = ddfc.MarkdownLine(tm_text2,text_align='center')
title_comp = ddfc.MarkdownDiv('mdiv1',[tm1,tm2])


### Create multiple dropdown div

In [ ]:
dfcf = df_iv_final[df_iv_final.commod != 'NG'][['settle_date','commod']]
dfcf['year'] = dfcf.settle_date.apply(lambda v: int(str(v)[0:4]))
dfcf = dfcf[['commod','year']]
df_all_years = pd.DataFrame({'commod':dfcf.commod.unique(),'year':'all'})
dfcf = df_all_years.append(dfcf,ignore_index=True)
mdd2 = ddfc.dgc.MultiDropdownDiv('mdd2',dfcf.copy(),['commod','year']) 
                                                 

### Create div using with multiple graphs using the VariableRowDiv class 
* In this example, 1 per row following title and dropdowns
* VariableRowDiv allows you to create a variable number of repeating rows, where you input:
 1. a list of component types that appear on each row
 2. a css grid spec for the row, that defines the percentage that each component takes on the row

In [ ]:
import importlib
importlib.reload(ddfc)
row_component_types = [ddfc.XYGraphSimple]  # a list of  component classs to use on each row
v = ddfc.VariableRowDiv('my_app',
            mdd2.dropdown_list,
            create_figs_closure(df_iv_final, df_iv_skew,df_cash_futures),
            row_component_types,
            '1fr',  # this is the css that defines how the components will look on each row
            logger=logger)


### Assemble the Dash app, using the title, the dropdown and the components from VariableRowDiv, and run the app
* ```VariableRowDiv.final_components``` contains all of the components you need to pass to ```make_app```
* ```VariableRowDiv.final_layout``` contains the css grid layout of the components you need to pass to ```make_app```

In [ ]:
ac = [title_comp,mdd2] + v.final_components
gc = ['1fr','1fr'] + v.final_layout 
app = ddfc.dgc.make_app(ac,grid_template_columns_list=gc)    
app.run_server(host='127.0.0.1',port=8500)    
